# Optimized PPO Inference on normal-Based Predictions

This notebook loads a pre-trained Optimized PPO (optimized from original two stage paper) model and applies it to the test set predictions generated by the base small LLM.

## Prerequisites:
- A trained PPO model saved at `../results/ppo_final_model.zip`
- Justification-based LLM predictions at `../results/llm_predictions_checkpoint.json`

## What this notebook does:
1. Loads the justification-based LLM predictions for the test set.
2. Loads the pre-trained PPO model.
3. Defines the RL environment required for inference.
4. Applies the PPO model to adjust the LLM's predictions.
5. Evaluates and saves the final, adjusted predictions.

## 1. Import Libraries

In [73]:
# Install required packages for progress bar
!pip install "stable-baselines3[extra]"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [74]:
# Import libraries
import os
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Reinforcement Learning
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Sentiment Analysis
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Progress bar
from tqdm import tqdm

# Set random seeds for reproducibility
np.random.seed(42)

def init_vader():
    """Downloads the VADER lexicon if not already present."""
    try:
        nltk.data.find('sentiment/vader_lexicon.zip')
    except LookupError:
        print("Downloading VADER lexicon...")
        nltk.download('vader_lexicon', quiet=True)
        print("Download complete.")

init_vader()
print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Data Loading and Preparation

In [75]:
def safe_float(value, default=0.0) -> float:
    try:
        return float(value)
    except (TypeError, ValueError):
        return float(default)

def parse_prompt_data(prompt_text):
    """Extract key information from prompt, including raw headlines."""
    lines = prompt_text.split('\n')
    data = {}
    
    for line in lines:
        if 'TICKER:' in line:
            data['ticker'] = line.split('TICKER:')[1].strip()
        elif 'DATE:' in line:
            data['date'] = line.split('DATE:')[1].strip()
        elif 'RECENT CLOSING PRICES' in line:
            if ':' in line:
                prices_part = line.split(':', 1)[1].strip()
                if '(' in prices_part:
                    prices_part = prices_part.split('(')[0].strip()
                try:
                    data['recent_prices'] = [float(p.strip()) for p in prices_part.split(',') if p.strip()]
                except ValueError:
                    data['recent_prices'] = []
        elif 'HEADLINES (concise):' in line:
            # The headlines are on the line immediately following this one
            try:
                # Find the index of the current line and get the next one
                current_line_index = lines.index(line)
                if current_line_index + 1 < len(lines):
                    data['headlines'] = lines[current_line_index + 1].strip()
            except:
                data['headlines'] = ""
    
    return data

print("Data preparation functions defined.")

Data preparation functions defined.


In [76]:
def load_jsonl(filepath):
    """Load JSONL file"""
    data = []
    with open(filepath, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [77]:
# Load the pre-computed LLM predictions from the checkpoint file
checkpoint_file = '../results/llm_predictions_checkpoint.json'
print(f"Loading pre-computed LLM predictions from {checkpoint_file}...")

try:
    # Load original test data to get prompts
    test_data = load_jsonl('../finetune_paper/test.jsonl')
    
    with open(checkpoint_file, 'r') as f:
        checkpoint_data = json.load(f)
    
    llm_results = checkpoint_data.get('llm_results', [])
    
    if len(llm_results) != len(test_data):
        raise ValueError(f"Checkpoint size ({len(llm_results)}) does not match test data size ({len(test_data)})")

    # Prepare the test DataFrame
    parsed_data = []
    for idx, item in enumerate(test_data):
        parsed = parse_prompt_data(item['prompt'])
        response = json.loads(item['response'])
        llm_output = llm_results[idx]

        parsed['llm_prediction'] = safe_float(llm_output.get('predicted_close'), response['predicted_close'])
        parsed['actual_price'] = response['predicted_close']
        parsed['llm_likelihood'] = safe_float(llm_output.get('likelihood'), 0.5)
        parsed_data.append(parsed)

    test_df = pd.DataFrame(parsed_data)
    
    # --- Perform Sentiment Analysis On-the-Fly ---
    sia = SentimentIntensityAnalyzer()
    
    def analyze_headlines(text):
        if not isinstance(text, str) or not text:
            return 0, 0.0
        
        headlines = text.split(' | ')
        if not headlines:
            return 0, 0.0
            
        scores = [sia.polarity_scores(h)['compound'] for h in headlines]
        return len(headlines), np.mean(scores)

    sentiment_scores = test_df['headlines'].apply(analyze_headlines)
    test_df['headline_count'] = sentiment_scores.apply(lambda x: x[0])
    test_df['sent_compound_mean'] = sentiment_scores.apply(lambda x: x[1])
    
    # Drop the raw headlines column as it's no longer needed
    test_df.drop(columns=['headlines'], inplace=True)

    print(f"Successfully loaded and prepared {len(test_df)} test samples.")
    display(test_df.head())

except FileNotFoundError:
    print(f"ERROR: Checkpoint file not found at {checkpoint_file}")
    test_df = pd.DataFrame()
except Exception as e:
    print(f"An error occurred: {e}")
    test_df = pd.DataFrame()

Loading pre-computed LLM predictions from ../results/llm_predictions_checkpoint.json...
Successfully loaded and prepared 2477 test samples.
Successfully loaded and prepared 2477 test samples.


,ticker,date,recent_prices,llm_prediction,actual_price,llm_likelihood,headline_count,sent_compound_mean
0,HSBC,2023-01-03,"[31.07, 31.03, 31.21, 31.16, 31.63]",31.63,32.680000,0.8,4,0.072325
1,0700.HK,2023-01-03,"[304.1191, 309.8178, 318.3658, 317.226, 327.8636]",0.00,342.870056,0.0,3,0.106067
2,PEP,2023-01-03,"[183.07, 181.75, 181.98, 180.66, 179.41]",181.00,178.970001,0.7,1,0.000000
3,AAPL,2023-01-03,"[130.03, 126.04, 129.61, 129.93, 125.07]",130.03,126.360001,0.5,5,0.068000
4,7203.T,2023-01-04,"[1817.5, 1819.0, 1817.0, 1812.5, 1799.0]",1817.50,1807.500000,0.8,7,0.035543


In [78]:
# Financial Risk Metrics
def calculate_var(returns: np.ndarray, confidence_level: float = 0.95) -> float:
    """Calculate Value at Risk (VaR)"""
    if len(returns) == 0:
        return 0.0
    return np.percentile(returns, (1 - confidence_level) * 100)

def calculate_cvar(returns: np.ndarray, confidence_level: float = 0.95) -> float:
    """Calculate Conditional Value at Risk (CVaR) - Expected Shortfall"""
    if len(returns) == 0:
        return 0.0
    var = calculate_var(returns, confidence_level)
    tail_losses = returns[returns <= var]
    if len(tail_losses) == 0:
        return var
    return np.mean(tail_losses)

def calculate_volatility(prices: np.ndarray) -> float:
    """Calculate price volatility (standard deviation of returns)"""
    if len(prices) < 2:
        return 0.0
    returns = np.diff(prices) / prices[:-1]
    return np.std(returns)

print("Risk metrics functions defined.")

Risk metrics functions defined.


## 3. PPO Environment Definition

In [79]:
# Custom Gym Environment for Stock Price Prediction with PPO
class StockPredictionEnv(gym.Env):
    """Custom Environment for Risk-Aware Stock Price Prediction without justification features"""
    
    def __init__(self, data_df: pd.DataFrame, window_size: int = 5):
        super(StockPredictionEnv, self).__init__()
        
        self.data = data_df.copy()
        self.window_size = window_size
        self.current_step = 0
        self.max_steps = len(self.data)
        
        # State: [llm_prediction, historical_prices (window), volatility, var, llm_likelihood]
        state_dim = 1 + window_size + 2 + 1  # 1 for llm_likelihood
        
        # Action space: adjustment factor (continuous)
        self.action_space = spaces.Box(
            low=-0.02, high=0.02, shape=(1,), dtype=np.float32
        )
        
        # Observation space
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(state_dim,), dtype=np.float32
        )
        
        # Risk parameters
        self.lambda_risk = 5.0
        self.confidence_level = 0.95
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        return self._get_observation(), {}
    
    def _get_observation(self):
        """Construct state representation with NaN handling"""
        idx = min(self.current_step, self.max_steps - 1)
        
        llm_pred = float(self.data.iloc[idx]['llm_prediction'])
        if np.isnan(llm_pred) or np.isinf(llm_pred):
            llm_pred = float(self.data.iloc[idx]['actual_price'])
        
        hist_prices = []
        if 'recent_prices' in self.data.columns and self.data.iloc[idx]['recent_prices'] is not None:
            try:
                hist_prices = [float(p) for p in self.data.iloc[idx]['recent_prices']]
                hist_prices = [p if not (np.isnan(p) or np.isinf(p)) else llm_pred for p in hist_prices]
            except:
                hist_prices = []
        
        if len(hist_prices) < self.window_size:
            pad_value = hist_prices[-1] if hist_prices else llm_pred
            hist_prices = hist_prices + [pad_value] * (self.window_size - len(hist_prices))
        hist_prices = np.array(hist_prices[-self.window_size:], dtype=np.float32)
        
        volatility = calculate_volatility(hist_prices)
        if np.isnan(volatility) or np.isinf(volatility): volatility = 0.0
        
        returns = np.diff(hist_prices) / hist_prices[:-1] if len(hist_prices) > 1 else np.array([0.0])
        returns = np.nan_to_num(returns, nan=0.0, posinf=0.0, neginf=0.0)
        var = calculate_var(returns, self.confidence_level)
        if np.isnan(var) or np.isinf(var): var = 0.0
        
        llm_likelihood = float(self.data.iloc[idx].get('llm_likelihood', 0.5))
        if np.isnan(llm_likelihood) or np.isinf(llm_likelihood): llm_likelihood = 0.5
        
        state = np.concatenate([
            np.array([llm_pred], dtype=np.float32),
            hist_prices,
            np.array([volatility, var, llm_likelihood], dtype=np.float32)
        ])
        
        state = np.nan_to_num(state, nan=0.0, posinf=1e6, neginf=-1e6)
        return state.astype(np.float32)
    
    def step(self, action):
        idx = min(self.current_step, self.max_steps - 1)
        llm_pred = float(self.data.iloc[idx]['llm_prediction'])
        actual_price = float(self.data.iloc[idx]['actual_price'])
        if np.isnan(llm_pred) or np.isinf(llm_pred): llm_pred = actual_price
        if np.isnan(actual_price) or np.isinf(actual_price): actual_price = llm_pred
        adjustment = float(action[0])
        if np.isnan(adjustment) or np.isinf(adjustment): adjustment = 0.0
        adjusted_pred = llm_pred * (1 + adjustment)
        pred_error = abs(adjusted_pred - actual_price)
        if actual_price != 0 and not np.isnan(actual_price): pct_error = pred_error / abs(actual_price)
        else: pct_error = 0.0
        if np.isnan(pred_error) or np.isinf(pred_error): pred_error = 0.0
        if np.isnan(pct_error) or np.isinf(pct_error): pct_error = 0.0
        scaled_error = pct_error * 100
        cvar = 0.0
        if 'recent_prices' in self.data.columns and self.data.iloc[idx]['recent_prices'] is not None:
            try:
                hist_prices = np.array(self.data.iloc[idx]['recent_prices'][-self.window_size:], dtype=np.float32)
                hist_prices = np.nan_to_num(hist_prices, nan=llm_pred)
                returns = np.diff(hist_prices) / hist_prices[:-1] if len(hist_prices) > 1 else np.array([0.0])
                returns = np.nan_to_num(returns, nan=0.0, posinf=0.0, neginf=0.0)
                cvar = abs(calculate_cvar(returns, self.confidence_level))
                if np.isnan(cvar) or np.isinf(cvar): cvar = 0.0
            except:
                cvar = 0.0
        risk_penalty = self.lambda_risk * cvar * 100
        llm_error = abs(llm_pred - actual_price)
        if actual_price != 0 and not np.isnan(actual_price): llm_pct_error = llm_error / abs(actual_price) * 100
        else: llm_pct_error = 0.0
        improvement = llm_pct_error - scaled_error
        reward = -scaled_error - risk_penalty + (improvement * 0.5)
        if np.isnan(reward) or np.isinf(reward): reward = -100.0
        self.current_step += 1
        terminated = self.current_step >= self.max_steps
        truncated = False
        next_state = self._get_observation()
        return next_state, reward, terminated, truncated, {}

print("Stock Prediction Environment defined.")

Stock Prediction Environment defined.


## 4. Load Pre-Trained PPO Model

In [80]:
# Load the saved PPO model
try:
    model_path = "../results/ppo_paper_final_model.zip"
    model = PPO.load(model_path)
    print(f"Successfully loaded PPO model from {model_path}")
except Exception as e:
    print(f"Error loading PPO model: {e}")
    model = None

Successfully loaded PPO model from ../results/ppo_paper_final_model.zip


In [81]:
# --- Define Environment and Risk Metrics ---

def calculate_var(returns: np.ndarray, confidence_level: float = 0.95) -> float:
    if len(returns) == 0: return 0.0
    return np.percentile(returns, (1 - confidence_level) * 100)

def calculate_cvar(returns: np.ndarray, confidence_level: float = 0.95) -> float:
    if len(returns) == 0: return 0.0
    var = calculate_var(returns, confidence_level)
    tail_losses = returns[returns <= var]
    return np.mean(tail_losses) if len(tail_losses) > 0 else var

class PaperStockEnv(gym.Env):
    def __init__(self, data_df: pd.DataFrame, window_size: int = 5, **kwargs):
        super(PaperStockEnv, self).__init__()
        self.data = data_df.copy()
        self.window_size = window_size
        self.current_step = 0
        self.max_steps = len(self.data)
        self.action_space = spaces.Discrete(5)
        self.action_map = {0: -0.02, 1: -0.01, 2: 0.0, 3: 0.01, 4: 0.02}
        # Correct state_dim with sentiment features
        state_dim = 1 + window_size + 2 + 1 + 2
        self.observation_space = spaces.Box(low=-5.0, high=5.0, shape=(state_dim,), dtype=np.float32)
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        return self._get_observation(), {}
    
    def _get_observation(self):
        idx = min(self.current_step, self.max_steps - 1)
        row = self.data.iloc[idx]
        recent_prices = row.get('recent_prices', [])
        if not isinstance(recent_prices, list) or len(recent_prices) == 0:
            recent_prices = [row['actual_price']] * self.window_size
        
        last_close = recent_prices[-1]
        llm_pred = row['llm_prediction']
        llm_pred_pct_change = (llm_pred - last_close) / last_close if last_close != 0 else 0.0
        hist_prices_pct_change = [(p - last_close) / last_close if last_close != 0 else 0.0 for p in recent_prices]
        
        if len(hist_prices_pct_change) < self.window_size:
            hist_prices_pct_change += [0.0] * (self.window_size - len(hist_prices_pct_change))
            
        returns = np.diff(recent_prices) / recent_prices[:-1] if len(recent_prices) > 1 else np.array([0.0])
        volatility = np.std(returns)
        var = np.percentile(returns, 5) if len(returns) > 0 else 0.0
        
        # Include sentiment features
        headline_count = row.get('headline_count', 0.0)
        sent_compound_mean = row.get('sent_compound_mean', 0.0)
        
        state = np.concatenate([
            np.array([llm_pred_pct_change], dtype=np.float32),
            np.array(hist_prices_pct_change[-self.window_size:], dtype=np.float32),
            np.array([volatility, var, row.get('llm_likelihood', 0.5), headline_count, sent_compound_mean], dtype=np.float32)
        ])
        return np.nan_to_num(state, nan=0.0, posinf=5.0, neginf=-5.0).astype(np.float32)

    def step(self, action):
        self.current_step += 1
        terminated = self.current_step >= self.max_steps
        return self._get_observation(), 0.0, terminated, False, {}

print("Environment and risk functions defined.")

Environment and risk functions defined.


## 5. Apply PPO to Test Data

In [82]:
def apply_ppo_adjustment(model, test_df):
    """
    Apply PPO adjustments to the LLM predictions.
    """
    # Use the same environment class as in training
    env = PaperStockEnv(test_df, window_size=5)
    obs, _ = env.reset()
    
    adjusted_predictions = []
    
    for idx in range(len(test_df)):
        if idx < env.window_size:
            # For early samples, not enough history, so use LLM prediction as-is
            adjusted_predictions.append(test_df.iloc[idx]['llm_prediction'])
            continue
        
        # Drop extra sentiment features for PPO model compatibility
        obs_for_model = obs[:9] if obs.shape[0] > 9 else obs
        # Get PPO action
        action, _ = model.predict(obs_for_model, deterministic=True)
        
        # Map discrete action to adjustment value
        adjustment = env.action_map[int(action)]
        
        # Apply adjustment
        llm_pred = test_df.iloc[idx]['llm_prediction']
        adjusted_pred = llm_pred * (1 + adjustment)
        adjusted_predictions.append(adjusted_pred)
        
        # Step environment
        if idx < len(test_df) - 1:
            obs, _, terminated, _, _ = env.step(action)
            if terminated:
                break
    return adjusted_predictions

print("Applying PPO adjustments to test set...")
test_df['ppo_adjusted_prediction'] = apply_ppo_adjustment(model, test_df)
print("PPO adjustments applied!")

# Display results
display(test_df[['llm_prediction', 'ppo_adjusted_prediction', 'actual_price']].head())

Applying PPO adjustments to test set...
PPO adjustments applied!
PPO adjustments applied!


,llm_prediction,ppo_adjusted_prediction,actual_price
0,31.63,31.63,32.680000
1,0.00,0.00,342.870056
2,181.00,181.00,178.970001
3,130.03,130.03,126.360001
4,1817.50,1817.50,1807.500000


## 6. Save and Evaluate Test Results

In [83]:
# Save test predictions with PPO adjustments
test_df.to_csv('../results/test_predictions_with_ppo_optimized.csv', index=False)
print(f"Test predictions with PPO adjustments saved to ../results/test_predictions_with_ppo_optimized.csv")

# Quick comparison
llm_mae = np.mean(np.abs(test_df['llm_prediction'] - test_df['actual_price']))
ppo_mae = np.mean(np.abs(test_df['ppo_adjusted_prediction'] - test_df['actual_price']))

print(f"\nQuick Comparison:")
print(f"LLM MAE: {llm_mae:.4f}")
print(f"LLM-PPO MAE: {ppo_mae:.4f}")
print(f"Improvement: {((llm_mae - ppo_mae) / llm_mae * 100):.2f}%")

Test predictions with PPO adjustments saved to ../results/test_predictions_with_ppo_optimized.csv

Quick Comparison:
LLM MAE: 62.1152
LLM-PPO MAE: 62.3913
Improvement: -0.44%
